In [3]:
from functools import partial
import geopyspark as gps
import numpy as np
import fiona
import json
import pyproj

from pyspark import SparkContext
from colortools import Color

from shapely.geometry import mapping, shape
from shapely.ops import transform
from geonotebook.vis.geotrellis.render_methods import render_nlcd, single_band_render_from_color_map
from geonotebook.wrappers import TMSRasterData, VectorData

In [4]:
conf = gps.geopyspark_conf(master="local[*]", appName="NLCD Viewer")
sc = SparkContext(conf=conf)

In [5]:
catalog_uri = "s3://azavea-datahub/catalog"
layer_name = "nlcd-tms-epsg3857"

## Vieweing NLCD

In [ ]:
M.set_center(-120.32, 47.84, 7)

In [ ]:
nlcd_cmap = gps.ColorMap.nlcd_colormap()
nlcd_tms_server = gps.TMS.build((catalog_uri, layer_name), nlcd_cmap)

In [ ]:
M.add_layer(TMSRasterData(nlcd_tms_server), name="NLCD")

## Viewing reclassified tiles

In [ ]:
def reclass(tile):
    # Planted/Cultivated
    # See https://www.mrlc.gov/nlcd11_leg.php
    result = tile.copy()
    result[np.ma.where((80 <= tile) & (tile < 90))] = 1
    result[np.ma.where((tile  < 80) | (90 <= tile))] = 0
    return result
     
cmap = { 0 : "#00000000", 1: "#CA9146FF" }
cmap_render = single_band_render_from_color_map(cmap)

def reclass_render(tile):
    reclassed = reclass(tile[0])
    return cmap_render(reclassed)

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
M.add_layer(TMSRasterData(nlcd_tms_server), render_tile=reclass_render)

## Chattanooga geometry

In [ ]:
!curl -o /tmp/mask.json https://s3.amazonaws.com/chattademo/chatta_mask.json

In [6]:
project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:3857'))

txt = open('/tmp/mask.json').read()
js = json.loads(txt)
geom = shape(js)
center = geom.centroid
chatta_poly = transform(project, geom)

In [ ]:
chatta_poly

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
vd = VectorData("/tmp/mask.json")
name = "Outline"
M.add_layer(vd, name=name)

In [ ]:
M.set_center(center.x, center.y, 9)

## Fetching an RDD of NLCD masked to Chattanooga

In [7]:
query_rdd = gps.query(gps.LayerType.SPATIAL,
                      catalog_uri,
                      layer_name,
                      12,
                      query_geom=chatta_poly)

In [8]:
masked = query_rdd.mask([chatta_poly])

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
masked_tms_server = gps.TMS.build(masked.pyramid(), display=nlcd_cmap)

In [ ]:
M.add_layer(TMSRasterData(masked_tms_server), name="Masked NLCD")

## Reclassifying an RDD

In [9]:
reclassified = masked.reclassify({0: 1, 80: 2, 90: 1},
                                 int,
                                 gps.ClassificationStrategy.GREATER_THAN_OR_EQUAL_TO).repartition(150)

In [10]:
colors = gps.get_colors_from_colors(
    [Color("#CA9146FF"), Color("#00FFAA88")])

breaks = {
    1: colors[0],
    2: colors[1]
}

reclassified_cmap = gps.ColorMap.build(breaks)

In [11]:
# This works and displays fine
reclassified_tms_server = gps.TMS.build(reclassified.pyramid(), display=reclassified_cmap)

In [13]:
M.remove_layer(M.layers[0])

In [12]:
M.add_layer(TMSRasterData(reclassified_tms_server), name="Reclassified NLCD")

## Saving the reclassified layer locally

In [15]:
local_catalog_uri = "file:///tmp/catalog"
local_layer_name = "cultivated-land-cover"

In [10]:
for layer in reclassified.pyramid().levels.values():
    gps.write(local_catalog_uri, local_layer_name, layer)

## Viewing the local Layer

In [16]:
# This does not display anything
local_tms_server = gps.TMS.build((local_catalog_uri, local_layer_name), reclassified_cmap)

In [ ]:
M.remove_layer(M.layers[0])

In [17]:
M.add_layer(TMSRasterData(local_tms_server), name="Local NLCD")

In [ ]:
layers = [gps.query(gps.LayerType.SPATIAL, local_catalog_uri, local_layer_name, x) for x in range(0, 11)]

In [ ]:
read_in_pyramid = gps.Pyramid(layers)

In [ ]:
# This cannot display as well
server = gps.TMS.build(read_in_pyramid, reclassified_cmap)

In [ ]:
M.add_layer(TMSRasterData(server), name="Local NLCD")